In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from prettytable import PrettyTable

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import CIFAR10
import numpy as np
from torch.utils.data import DataLoader

In [27]:

# 1. Load the CIFAR-10 training dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
data = train_dataset.data

data = data / 255.0

mean = np.mean(data, axis=(0, 1, 2))

std = np.std(data, axis=(0, 1, 2))

# Print the results
print(f"Calculated Mean: {mean}")
print(f"Calculated Std Dev: {std}")

Calculated Mean: [0.49139968 0.48215841 0.44653091]
Calculated Std Dev: [0.24703223 0.24348513 0.26158784]


In [28]:
mean = [0.4914, 0.4822, 0.4465]
std = [0.2470, 0.2435, 0.2616]

train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.1),
    A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, fill_value=mean, mask_fill_value=None, p=0.1),
    A.Normalize(mean=mean, std=std),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Normalize(mean=mean, std=std),
    ToTensorV2(),
])



class AlbumentationDataset(CIFAR10):
    def __init__(self, root="./data", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label

train_dataset = AlbumentationDataset(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = AlbumentationDataset(root='./data', train=False, download=True, transform=test_transforms)



/usr/local/lib/python3.12/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipython-input-1763199312.py:7: UserWarning: Argument(s) 'max_holes, max_height, max_width, min_holes, min_height, min_width, fill_value, mask_fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16, fill_value=mean, mask_fill_value=None, p=0.1),


In [29]:
SEED = 2

# CUDA?
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
cuda = torch.cuda.is_available()
print("GPU Available?", device)

# For reproducibility
torch.manual_seed(SEED)

if device == "cuda":
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=128)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_dataset, **dataloader_args)

# Pretty table for collecting all the accuracy and loss parameters in a table
log_table = PrettyTable()


GPU Available? cuda


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [30]:
import torch.nn as nn

DROPOUT_PROB = 0.1

class CIFAR10Net(nn.Module):
    def __init__(self):
        super(CIFAR10Net, self).__init__()

        # BLOCK 1
        # Input: 32x32x3 | Output: 32x32x32 | RF: 5
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(DROPOUT_PROB),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(DROPOUT_PROB)
        )

        # BLOCK 2
        # Input: 32x32x32 | Output: 16x16x32 | RF: 7
        self.conv_block2 = nn.Sequential(
             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=2, padding=1, bias=False),
             nn.ReLU(),
             nn.BatchNorm2d(32)
        )

        # BLOCK 3
        # Input: 16x16x32 | Output: 16x16x64 | RF: 31
        self.conv_block3 = nn.Sequential(
            # Depthwise Separable Convolution
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, groups=32, bias=False), # Depthwise
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=1, bias=False), # Pointwise
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(DROPOUT_PROB),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=2, dilation=2, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(DROPOUT_PROB),
        )

        # BLOCK 4
        # Input: 16x16x64 | Output: 8x8x128 | RF: 39
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(128)
        )

        # OUTPUT BLOCK
        # Input: 8x8x128 | Output: 1x1x10 | RF: 55
        self.output_block = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # GAP layer
            nn.Conv2d(in_channels=128, out_channels=10, kernel_size=1, bias=False)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.output_block(x)
        x = x.view(-1, 10)
        return x



In [31]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
cuda = torch.device("cuda" if use_cuda else "cpu")
print(cuda)
model = CIFAR10Net().to(cuda)
summary(model, input_size=(3, 32, 32))


cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
           Dropout-4           [-1, 16, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]           4,608
              ReLU-6           [-1, 32, 32, 32]               0
       BatchNorm2d-7           [-1, 32, 32, 32]              64
           Dropout-8           [-1, 32, 32, 32]               0
            Conv2d-9           [-1, 32, 16, 16]           9,216
             ReLU-10           [-1, 32, 16, 16]               0
      BatchNorm2d-11           [-1, 32, 16, 16]              64
           Conv2d-12           [-1, 32, 16, 16]             288
           Conv2d-13           [-1, 64, 16, 16]           2,048
             ReLU-14           [-1

In [32]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    criterion = nn.CrossEntropyLoss()
    loss = criterion(y_pred, target)
    #loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss(reduction='sum')

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))


In [33]:
from torch.optim.lr_scheduler import StepLR
from prettytable import PrettyTable, MSWORD_FRIENDLY, MARKDOWN

print("model running on: ", device)
log_table = PrettyTable()
log_table.field_names = ["Epoch", "Training Accuracy", "Test Accuracy", "Diff", "Training Loss", "Test Loss"]

model =  CIFAR10Net().to(device)
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.003)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)

EPOCHS = 50
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    #scheduler.step()
    test(model, device, test_loader)
    log_table.add_row([epoch+1, f"{train_acc[-1]:.2f}%", f"{test_acc[-1]:.2f}%", f"{float(train_acc[-1]) - float(test_acc[-1]):.2f}" ,f"{train_losses[-1]:.4f}", f"{test_losses[-1]:.4f}"])
log_table.set_style(MARKDOWN)
print(log_table)


/tmp/ipython-input-1556196367.py:2: DeprecationWarning: the 'MSWORD_FRIENDLY' constant is deprecated, use the 'TableStyle' enum instead
  from prettytable import PrettyTable, MSWORD_FRIENDLY, MARKDOWN
/tmp/ipython-input-1556196367.py:2: DeprecationWarning: the 'MARKDOWN' constant is deprecated, use the 'TableStyle' enum instead
  from prettytable import PrettyTable, MSWORD_FRIENDLY, MARKDOWN


model running on:  cuda
EPOCH: 1


Loss=1.223803162574768 Batch_id=390 Accuracy=49.58: 100%|██████████| 391/391 [00:10<00:00, 35.99it/s]



Test set: Average loss: 1.0870, Accuracy: 6057/10000 (60.57%)

EPOCH: 2


Loss=0.9151042699813843 Batch_id=390 Accuracy=64.34: 100%|██████████| 391/391 [00:10<00:00, 38.26it/s]


...


Test set: Average loss: 0.4761, Accuracy: 8478/10000 (84.78%)

EPOCH: 49


Loss=0.3766241669654846 Batch_id=390 Accuracy=89.77: 100%|██████████| 391/391 [00:10<00:00, 37.62it/s]



Test set: Average loss: 0.4630, Accuracy: 8540/10000 (85.40%)

EPOCH: 50


Loss=0.41594618558883667 Batch_id=390 Accuracy=90.05: 100%|██████████| 391/391 [00:10<00:00, 35.83it/s]



Test set: Average loss: 0.4559, Accuracy: 8538/10000 (85.38%)

| Epoch | Training Accuracy | Test Accuracy |  Diff  | Training Loss | Test Loss |
| :---: | :---------------: | :-----------: | :----: | :-----------: | :-------: |
|   1   |       49.58%      |     60.57%    | -10.99 |     1.2238    |   1.0870  |
|   2   |       64.34%      |     68.00%    | -3.66  |     0.9151    |   0.9155  |
|   3   |       70.18%      |     72.81%    | -2.63  |     0.9749    |   0.7707  |
|   4   |       73.57%      |     74.77%    | -1.20  |     0.7730    |   0.7294  |
|   5   |       76.14%      |     76.25%    | -0.11  |     0.5815    |   0.6831  |
|   6   |       77.98%      |     78.90%    | -0.92  |     0.8226    |   0.6047  |
|   7   |       79.31%      |     79.40%    | -0.09  |     0.7528    |   0.5960  |
|   8   |       80.24%      |     79.88%    |  0.36  |     0.6128    |   0.5838  |
|   9   |       81.17%      |     81.29%    | -0.12  |     0.6029    |   0.5375  |
|   10  |       81.67% 